In [6]:
# define langchain tool
import os
from apikey import OPENAI_API_KEY
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, Tool

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

def analyze_text(text:str):
    llm = ChatOpenAI(model="gpt-3.5-turbo-1106")
    prompt_template = """
    Analyze the provided steel specification document and extract detailed information about each steel grade mentioned. Focus on identifying specific characteristics, standards, and requirements associated with each grade. Summarize the findings in a concise and structured format.
    """
    prompt = ChatPromptTemplate.from_template(prompt_template)
    analyze_text_chain = RunnableParallel({"query":RunnablePassthrough()}) | prompt | llm

    return analyze_text_chain.invoke({"query":text})

def llm_generator(text:str):
    llm = ChatOpenAI(model="gpt-3.5-turbo-1106")
    
    return llm.invoke(text)

# Text 참조해서 topic Analyze 하는 tool
topic_info_tool = Tool.from_function(
    func=analyze_text,
    name="GetMultipleSteelInfoWithText",
    description="Analyzes a given text string about Steel and returns various related information"
)

# 상관없는 다른 주제라면 llm_generator 이용
common_sense_tool = Tool.from_function(
    func = llm_generator,
    name="CommonSenseGenerator",
    description="Generate common sense answer, not specific Steel topic."
)

In [7]:
steel_spec_text = """GENERAL MATERIAL
SPECIFICATION
NO:   MS-1001 DRAFT
REV:  10
TITLE:   STEEL BAR, BILLET, BLOOM, ROD, WIRE, SEAMLESS
TUBE & FORGINGS
DATE:  10-Apr-2018
PAGE:  1 of 11
REVISED BY:   M. Binno, P.E., Tech Mgr, Materials Engineering
FINAL APPROVAL:   S. Cook, Director, Materials Engineering
1.0 PURPOSE :    This standard outlines the general requirements for steel used by AAM or its suppliers in the  manufacture of AAM components.    2.0 SCOPE:     This standard covers the general guidelines for steelmaking, deoxidation, ladle metallurgy,  cleanliness, grain refining, vacuum degassing, casting, rolling and inspection practices for Special  Bar Quality (SBQ) bars used in the manufacture of AAM products.    Additional requirements for specific steel grades and end uses are covered by an AAM Material  Specification (MS-XXXX-X or MS-XXXX).    If this specification and the individual MS specification or part drawing are in conflict, then the  individual MS specification or part drawing takes precedence.    Revisions to this specification are required in writing to the Director or designee of AAM Corporate  Materials Engineering.    3.0 REFERENCES:
Specification  Title
ASTM A255  Standard Test Methods for Determining Hardenability of Steel
ASTM E45  Standard Test Methods for Determining the Inclusion Content of Steel     
ASTM E112  Standard Test Methods for Determining Average Grain Size
ASTM E381  Standard Test Method of Macroetch Testing Steel Bars, Billets, Blooms,  and Forgings
ASTM E588  Standard Practice for Detection of Large Inclusions in Bearing Quality  Steel by the Ultrasonic Method
ASTM E1077  Standard Test Methods for Estimating the Depth of Decarburization of  Steel Specimens
ASTM E1180  Standard Practice for Preparing Sulfur Prints for Macrostructural  Evaluation
ASTM E1268  Standard Practice for Assessing the Degree of Banding or Orientation  of Microstructures
Caterpillar 1E0024  Wrought Steel – General Requirements
EN 10060  Hot Rolled Round Steel Bars for General Purposes - Dimensions and  Tolerances on Shape and Dimensions
ISO 642  Steel - Hardenability Test by End Quenching (Jominy Test)
ISO 643  Steels - Micrographic Determination of the Apparent Grain Size
ISO/TS 16949
Quality Management Systems - Particular Requirements for the  Application of ISO 9001:2008 for Automotive Production and Relevant  Service Part Organizations
QS-9000  Quality System Requirements
SAE EA-406  Hardenability Predictor
SAE J406  Methods of Determining Hardenability of Steels
SAE J1268  Hardenability Bands for Carbon and Alloy H Steels"""

In [10]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_community.tools.convert_to_openai import format_tool_to_openai_function

# Initialize llm 
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

# 에이전트를 위한 프롬프트 템플릿 생성
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "철강 사양을 분석하는 보조원입니다. 만약 다른 주제를 입력하면, 일반적인 답변도 들려줄 수 있습니다."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm_with_tool = llm.bind(functions=[format_tool_to_openai_function(topic_info_tool), format_tool_to_openai_function(common_sense_tool)])

# 에이전트 생성
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(x["intermediate_steps"]),
    }
    | prompt_template
    | llm_with_tool
    | OpenAIFunctionsAgentOutputParser()
)

# 에이전트 초기화
agent_executor = AgentExecutor(agent=agent, tools=[topic_info_tool], verbose=True)

# 에이전트 실행
# result = agent_executor.invoke({"input": steel_spec_text})


# result_text = result



> Entering new AgentExecutor chain...

Invoking: `TopicInfo` with `STEEL BAR, BILLET, BLOOM, ROD, WIRE, SEAMLESS TUBE & FORGINGS`


content='Steel Grade: A36\n- Characteristics: Low carbon steel, good weldability, and machinability\n- Standards: ASTM A36/A36M-08\n- Requirements: Tensile strength of 400-550 MPa, yield strength of 250 MPa, elongation of 20% in 2 inches\n\nSteel Grade: A572\n- Characteristics: High strength, low alloy steel, good formability\n- Standards: ASTM A572/A572M-18\n- Requirements: Tensile strength of 450-620 MPa, yield strength of 345 MPa, elongation of 18% in 2 inches\n\nSteel Grade: A514\n- Characteristics: Quenched and tempered alloy steel, high yield strength\n- Standards: ASTM A514/A514M-18\n- Requirements: Tensile strength of 760-895 MPa, yield strength of 690 MPa, elongation of 18% in 2 inches\n\nSteel Grade: A709\n- Characteristics: High strength, low alloy steel, good weldability\n- Standards: ASTM A709/A709M-18\n- Requirements: Tensile strength of 48

In [13]:
print(result_text['output'])

The steel specification document provides detailed information about various steel grades, including their specific characteristics, standards, and requirements. Here are some of the steel grades mentioned in the document:

1. Steel Grade: A36
   - Characteristics: Low carbon steel, good weldability, and machinability
   - Standards: ASTM A36/A36M-08
   - Requirements: Tensile strength of 400-550 MPa, yield strength of 250 MPa, elongation of 20% in 2 inches

2. Steel Grade: A572
   - Characteristics: High strength, low alloy steel, good formability
   - Standards: ASTM A572/A572M-18
   - Requirements: Tensile strength of 450-620 MPa, yield strength of 345 MPa, elongation of 18% in 2 inches

3. Steel Grade: A514
   - Characteristics: Quenched and tempered alloy steel, high yield strength
   - Standards: ASTM A514/A514M-18
   - Requirements: Tensile strength of 760-895 MPa, yield strength of 690 MPa, elongation of 18% in 2 inches

4. Steel Grade: A709
   - Characteristics: High strength,